In [1]:
%tensorflow_version 2.x
%load_ext tensorboard 

import tensorflow as tf
from tensorflow.keras import layers, datasets, utils, backend, Model
import datetime
from sklearn.metrics import roc_curve
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import pathlib
import time
import seaborn as sns
import numpy as np
import time
!rm -rf ./logs/

#tf.config.run_functions_eagerly(True)

In [2]:
def img_rmse(image1, image2):
  error = np.sqrt(np.sum((image1.astype('float')-image2.astype('float'))**2))
  return error

# Load MNIST and Fashion datasets
mn = tf.keras.datasets.mnist
fs = tf.keras.datasets.fashion_mnist
(mx_train, my_train), (mx_test, my_test) = mn.load_data() # MNIST
(fx_train, fy_train), (fx_test, fy_test) = fs.load_data() # Fashion MNIST

# Normalize Image values to [0,1]
mx_train = mx_train/255.0
mx_test = mx_test/255.0
fx_train = fx_train/255.0
fx_test = fx_test/255.0
# One Hot Encode the categories, num_classes = 10
my_train = utils.to_categorical(my_train)
my_test = utils.to_categorical(my_test)
fy_train = utils.to_categorical(fy_train)
fy_test = utils.to_categorical(fy_test)
# Get Image Dimensions, store into img_size

img_w = mx_train.shape[1]
img_h = mx_train.shape[2]

# Get Image shape and number of labels in the classifier
img_shape = (img_w, img_h, 1)
num_labels = my_train.shape[1]

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
# Hyperparameters for Hybrid and Independent neural networks
input_shape = img_shape
batch_size = 64
kernel_size = (3,3)
pool_size = (2,2)
latent_dim = 64
filters = 64
layer_filters = [32, 64]
dropout = .25
Epochs = 20

In [4]:
# Build the Hybrid Network

# Core Network Layers
inputs = tf.keras.layers.Input(shape = input_shape, name = 'common_input')
x = inputs

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

core = tf.keras.layers.Dense(latent_dim, name = 'waist') (x)

# Classifier Network Sublayers
flat = (tf.keras.layers.Flatten())(core)
drop = (tf.keras.layers.Dropout(dropout))(flat)

dense = (tf.keras.layers.Dense(num_labels))(drop)
class_out = (tf.keras.layers.Activation('softmax', name = "Classifier"))(dense)

# Autoencoder Network Sublayers
x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(core)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

auto_out = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                      kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'Generator')(x)


In [5]:
# Build the Hybrid Network

# Core Network Layers
inputs2 = tf.keras.layers.Input(shape = input_shape, name = 'common_input')
x = inputs2

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', strides = 2, 
                             padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

core = tf.keras.layers.Dense(latent_dim, name = 'waist') (x)

# Classifier Network Sublayers
flat = (tf.keras.layers.Flatten())(core)
drop = (tf.keras.layers.Dropout(dropout))(flat)

dense = (tf.keras.layers.Dense(num_labels))(drop)
class_out2 = (tf.keras.layers.Activation('softmax', name = "Classifier2"))(dense)

# Autoencoder Network Sublayers
x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(core)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

auto_out2 = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                      kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'Generator2')(x)


In [6]:
# Build the Hybrid Network

# Core Network Layers
inputs3 = tf.keras.layers.Input(shape = input_shape, name = 'common_input')
x = inputs3

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', strides = 2, 
                             padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

core = tf.keras.layers.Dense(latent_dim, name = 'waist') (x)

# Classifier Network Sublayers
flat = (tf.keras.layers.Flatten())(core)
drop = (tf.keras.layers.Dropout(dropout))(flat)

dense = (tf.keras.layers.Dense(num_labels))(drop)
class_out3 = (tf.keras.layers.Activation('softmax', name = "Classifier3"))(dense)

# Autoencoder Network Sublayers
x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(core)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

auto_out3 = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                      kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'Generator3')(x)

In [7]:
# Build the Hybrid Network

# Core Network Layers
inputs4 = tf.keras.layers.Input(shape = input_shape, name = 'common_input')
x = inputs4

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', strides = 2, 
                             padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

core = tf.keras.layers.Dense(latent_dim, name = 'waist') (x)

# Classifier Network Sublayers
flat = (tf.keras.layers.Flatten())(core)
drop = (tf.keras.layers.Dropout(dropout))(flat)

dense = (tf.keras.layers.Dense(num_labels))(drop)
class_out4 = (tf.keras.layers.Activation('softmax', name = "Classifier4"))(dense)

# Autoencoder Network Sublayers
x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(core)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

auto_out4 = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                      kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'Generator4')(x)

In [8]:
# Build the Hybrid Network

# Core Network Layers
inputs5 = tf.keras.layers.Input(shape = input_shape, name = 'common_input')
x = inputs5

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', strides = 2, 
                             padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

core = tf.keras.layers.Dense(latent_dim, name = 'waist') (x)

# Classifier Network Sublayers
flat = (tf.keras.layers.Flatten())(core)
drop = (tf.keras.layers.Dropout(dropout))(flat)

dense = (tf.keras.layers.Dense(num_labels))(drop)
class_out5 = (tf.keras.layers.Activation('softmax', name = "Classifier5"))(dense)

# Autoencoder Network Sublayers
x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(core)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

auto_out5 = tf.keras.layers.Conv2DTranspose(filters = 1, 
                                      kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'Generator5')(x)

In [9]:

#  CL Hybrid, fully weighed to generator, should match Classifier
cl_Hyb = tf.keras.Model(inputs = inputs2,
                          outputs = [class_out2, auto_out2],)

cl_Hyb.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
    loss={
        "Classifier2": tf.keras.losses.categorical_crossentropy,
        "Generator2": tf.keras.losses.MSE,
    },
    loss_weights=[1,0],
       metrics = ['accuracy']    
)

#  25% Classifier, 75% Generator
a_Hyb = tf.keras.Model(inputs = inputs3,
                          outputs = [class_out3, auto_out3],)

a_Hyb.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
    loss={
        "Classifier3": tf.keras.losses.categorical_crossentropy,
        "Generator3": tf.keras.losses.MSE,
    },
    loss_weights=[.25, .75],
       metrics = ['accuracy']    
)

#  50% Classifier, 50% Generator
b_Hyb = tf.keras.Model(inputs = inputs4,
                          outputs = [class_out4, auto_out4],)

b_Hyb.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
    loss={
        "Classifier4": tf.keras.losses.categorical_crossentropy,
        "Generator4": tf.keras.losses.MSE,
    },
    loss_weights=[.5, .5],
       metrics = ['accuracy']    
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tb_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
# Build the independent classifier and autoencoder

# Build the Convolutional Neural Network

cnn = tf.keras.models.Sequential()
# First Layers
cnn.add(tf.keras.layers.Conv2D(filters = filters, kernel_size=kernel_size,
                               activation = 'relu', input_shape = input_shape))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size))
# Second Layers
cnn.add(tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size,
                               activation = 'relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size))
# 3rd Layers
cnn.add(tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size,
                               activation = 'relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(dropout))

cnn.add(tf.keras.layers.Dense(num_labels))
cnn.add(tf.keras.layers.Activation('softmax'))
#cnn.summary()
tf.keras.utils.plot_model (cnn, to_file = 'cnn_structure.png', 
                           show_shapes = True)
cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
            metrics = ['accuracy'])

################################################################################

# The Autoencoder Model

inputs = tf.keras.layers.Input(shape = input_shape, name = 'encoder_input')
x = inputs

for filters in layer_filters:
  x = tf.keras.layers.Conv2D(filters = filters, kernel_size= kernel_size,
                             activation = 'relu', strides = 2, 
                             padding = 'same')(x)
shape = tf.keras.backend.int_shape(x)
x = tf.keras.layers.Flatten()(x)

latent = tf.keras.layers.Dense(latent_dim, name = 'latent_vector') (x)

# Build the Encoder
encoder = tf.keras.Model(inputs, latent, name = 'encoder')
# encoder.summary()

tf.keras.utils.plot_model(encoder, to_file = 'encoder_structure.png', 
                          show_shapes = True)

# Build the Decoder
latent_inputs = tf.keras.layers.Input(shape = (latent_dim, ), 
                                      name = 'decoder_input')

x = tf.keras.layers.Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
x = tf.keras.layers.Reshape((shape[1], shape[2], shape[3]))(x)

for filters in layer_filters[::-1]:
  x = tf.keras.layers.Conv2DTranspose(filters = filters, 
                                      kernel_size = kernel_size,
                                      activation = 'relu', strides = 2, 
                                      padding = 'same')(x)

outputs = tf.keras.layers.Conv2DTranspose(filters = 1, kernel_size = kernel_size,
                                      activation = 'sigmoid', padding = 'same',
                                      name = 'decoder_output')(x)

decoder = tf.keras.Model(latent_inputs, outputs, name = 'decoder')
#decoder.summary()

tf.keras.utils.plot_model(decoder, to_file = 'decoder.png', show_shapes = True)

# Build the Autoencoder
autoencoder = tf.keras.Model(inputs, decoder(encoder(inputs)), 
                             name = 'autoencoder')
# autoencoder.summary()
tf.keras.utils.plot_model(autoencoder, to_file = 'autoencoder_summary.png',
                          show_shapes = True)

autoencoder.compile(loss='mse', optimizer = 'adam')
################################################################################


In [ ]:
ae_hyb_time = 0
cl_hyb_time = 0
a_hyb_time = 0
b_hyb_time = 0
c_hyb_time = 0
ind_time = 0

start = time.time()
cl_Hyb.fit(fx_train, 
          {"Classifier2" : fy_train, "Generator2": fx_train},
          epochs = Epochs,
          batch_size = batch_size,
          callbacks=[tb_cb],)
stop = time.time()
cl_hyb_time = stop-start

start = time.time()
a_Hyb.fit(fx_train, 
          {"Classifier3" : fy_train, "Generator3": fx_train},
          epochs = Epochs,
          batch_size = batch_size,
          callbacks=[tb_cb],)
stop = time.time()
a_hyb_time = stop-start

start = time.time()
b_Hyb.fit(fx_train, 
          {"Classifier4" : fy_train, "Generator4": fx_train},
          epochs = Epochs,
          batch_size = batch_size,
          callbacks=[tb_cb],)
stop = time.time()
b_hyb_time = stop-start
# reshape the datsets to accomodate the independent networks, renamed
mxr = np.reshape(fx_train, [-1, 28, 28, 1])
mxrt = np.reshape(fx_test, [-1, 28, 28, 1])

start = time.time()
cnn.fit(mxr, fy_train, epochs = Epochs, batch_size = batch_size,)
autoencoder.fit(fxr, fx_train, epochs = Epochs, batch_size = batch_size)
stop = time.time()
ind_time = stop - start

Epoch 1/20
570/938 [=================>............] - ETA: 2s - loss: 0.5493 - Classifier2_loss: 0.5493 - Generator2_loss: 0.1707 - Classifier2_accuracy: 0.8061 - Generator2_accuracy: 0.2471

In [ ]:

print("CL Hybrid Training Time: ", cl_hyb_time)
print("A Hybrid Training Time: ", a_hyb_time)
print("B Hybrid Training Time: ", b_hyb_time)

print("Independent Training Time: ", ind_time)

In [ ]:

ind_fpr = dict()
ind_tpr = dict()
chyb_fpr = dict()
chyb_tpr = dict()
ghyb_fpr = dict()
ghyb_tpr = dict()

c25_fpr = dict()
c25_tpr = dict()
c50_fpr = dict()
c50_tpr = dict()
c75_fpr = dict()
c75_tpr = dict()


# Create the merged test dataset and re evaluate 
x_test = np.append(mx_test, fx_test, axis = 0)
y_test = np.append(my_test, fy_test, axis = 0)
xtr = np.reshape(x_test, [-1, 28, 28, 1])





cbtime=0
gbtime=0
atime=0
btime=0
ctime=0
itime=0

start = time.time()
[CB_class, CB_gen] = cl_Hyb.predict(x_test) # Classifier Biased hybrid predictions
stop = time.time()
cbtime = stop-start


start = time.time()
[C25_class, C25_gen] = a_Hyb.predict(x_test) # 25% class / 75% gen bias
stop=time.time()
atime=stop-start

start=time.time()
[C50_class, C50_gen] = b_Hyb.predict(x_test) # 50% class / 50% gen bias
stop=time.time()
btime=stop-start


start=time.time()
ind_class = cnn.predict(xtr) # Independent classifier predictions
ind_gen = autoencoder.predict(x_test) # Independent Autoencoder generations
stop=time.time()
itime=stop-start

sep_loss = np.empty(len(x_test))
chyb_loss = np.empty(len(x_test))
ghyb_loss = np.empty(len(x_test))
c25_loss = np.empty(len(x_test))
c50_loss = np.empty(len(x_test))



for i in range(len(x_test)):
  sep_loss[i] = img_rmse(x_test[i], np.squeeze(ind_gen[i]))
  chyb_loss[i] = img_rmse(x_test[i], np.squeeze(CB_gen[i]))
  c25_loss[i] = img_rmse(x_test[i], np.squeeze(C25_gen[i]))
  c50_loss[i] = img_rmse(x_test[i], np.squeeze(C50_gen[i]))


for i in range(15):
  print('Classifier Biased Prediction: ',np.argmax(CB_class[i]),
        '\n Generator Biased Prediction: ',np.argmax(GB_class[i]),
        '\n 25% Classifier Biased Prediction: ',np.argmax(C25_class[i]),
        '\n 50% Classifier Biased Prediction: ',np.argmax(C50_class[i]),
        '\n 75% Classifier Biased Prediction: ',np.argmax(C75_class[i]),
        '\n Independent Network Prediction: ', np.argmax(ind_class[i]),
        '\n Actual Value: ', np.argmax(y_test[i]))
  
  print(' Independent Network RMSE: ', sep_loss[i],
        '\n Classifier Biased RMSE: ', chyb_loss[i],
        '\n Generator Biased RMSE: ', ghyb_loss[i],
        '\n 25% Classifier Biased RMSE: ', c25_loss[i],
        '\n 50% Classifier Biased RMSE: ', c50_loss[i],
        '\n 75% Classifier Biased RMSE: ', c75_loss[i])


for i in range(10):
    ind_fpr[i], ind_tpr[i], _ = roc_curve(y_test[:,i], ind_class[:, i])
    chyb_fpr[i], chyb_tpr[i], _ = roc_curve(y_test[:, i], CB_class[:, i])
    ghyb_fpr[i], ghyb_tpr[i], _ = roc_curve(y_test[:, i], GB_class[:, i])
    c25_fpr[i], c25_tpr[i], _ = roc_curve(y_test[:,i], C25_class[:, i])
    c50_fpr[i], c50_tpr[i], _ = roc_curve(y_test[:, i], C50_class[:, i])
    c75_fpr[i], c75_tpr[i], _ = roc_curve(y_test[:, i], C75_class[:, i])
ind_fpr["micro"], ind_tpr["micro"], _ = roc_curve(y_test.ravel(), ind_class.ravel())
chyb_fpr["micro"], chyb_tpr["micro"], _ = roc_curve(y_test.ravel(), CB_class.ravel())
ghyb_fpr["micro"], ghyb_tpr["micro"], _ = roc_curve(y_test.ravel(), GB_class.ravel())
c25_fpr["micro"], c25_tpr["micro"], _ = roc_curve(y_test.ravel(), C25_class.ravel())
c50_fpr["micro"], c50_tpr["micro"], _ = roc_curve(y_test.ravel(), C50_class.ravel())
c75_fpr["micro"], c75_tpr["micro"], _ = roc_curve(y_test.ravel(), C75_class.ravel())

plt.figure()
for i in range(10):
  plt.plot(ind_fpr[i], ind_tpr[i],color = 'blue')
  plt.plot(chyb_fpr[i], chyb_tpr[i], color = 'coral')
  plt.plot(ghyb_fpr[i], ghyb_tpr[i], color = 'forestgreen')
  plt.plot(c25_fpr[i], c25_tpr[i],color = 'black')
  plt.plot(c50_fpr[i], c50_tpr[i], color = 'red')
  plt.plot(c75_fpr[i], c75_tpr[i], color = 'orange')  
plt.plot([0,1], [0,1], 'k--')
plt.xlim([-.05, 1.0])
plt.ylim([0.0, 1.05])
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

plt.figure()
plt.plot(ghyb_fpr["micro"], ghyb_tpr["micro"], color = 'forestgreen', label ='Gen. Bias')
plt.plot(c25_fpr["micro"], c25_tpr["micro"], color = 'black', label = '25% Classifier Biased ')
plt.plot(c50_fpr["micro"], c50_tpr["micro"], color = 'red', label = '50% Classifier Biased')
plt.plot(c75_fpr["micro"], c75_tpr["micro"], color = 'orange', label ='75% Classifier Biased')
plt.plot(ind_fpr["micro"], ind_tpr["micro"], color = 'blue', label = 'Independent')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
print(np.average(sep_loss[:10000]))
print(np.average(chyb_loss[:10000]))
print(np.average(ghyb_loss[:10000]))
print(np.average(c25_loss[:10000]))
print(np.average(c50_loss[:10000]))
print(np.average(c75_loss[:10000]))
print()
print(np.average(sep_loss[10001:]))
print(np.average(chyb_loss[10001:]))
print(np.average(ghyb_loss[10001:]))
print(np.average(c25_loss[10001:]))
print(np.average(c50_loss[10001:]))
print(np.average(c75_loss[10001:]))


print()
print()
print("AE Hybrid Evaluation Time: {}", cbtime)
print("CL Hybrid Evaluation Time: {}", gbtime)
print("A Hybrid Evaluation Time: {}", atime)
print("B Hybrid Evaluation Time: {}", btime)
print("C Hybrid Evaluation Time: {}", ctime)
print("Independent Evaluation Time: {}", itime)

In [ ]:
# Create Image and Label holding arrays for Watchdog
aimg = []
bimg = []
cimg = []
dimg = []
eimg = []
iimg = []
alab = []
blab = []
clab = []
dlab = []
elab = []
ilab = []

# Create empty dictionaries for ROC curve FPR/TPR
afpr = dict()
bfpr = dict()
cfpr = dict()
dfpr = dict()
efpr = dict()
ifpr = dict()
atpr = dict()
btpr = dict()
ctpr = dict()
dtpr = dict()
etpr = dict()
itpr = dict()

thresh = 4.5

for i in range(len(x_test)):
  real = x_test[i]
  ap = np.squeeze(CB_gen[i])
  bp = np.squeeze(GB_gen[i])
  cp = np.squeeze(C25_gen[i])
  dp = np.squeeze(C50_gen[i])
  ep = np.squeeze(C75_gen[i])
  ip = np.squeeze(ind_gen[i])

  if(img_rmse(real, ap) < thresh):
    aimg.append(x_test[i])
    alab.append(y_test[i])

  if(img_rmse(real, bp) < thresh):
    bimg.append(x_test[i])
    blab.append(y_test[i])

  if(img_rmse(real, cp) < thresh):
    cimg.append(x_test[i])
    clab.append(y_test[i])

  if(img_rmse(real, dp) < thresh):
    dimg.append(x_test[i])
    dlab.append(y_test[i])

  if(img_rmse(real, ep) < thresh):
    eimg.append(x_test[i])
    elab.append(y_test[i])

  if(img_rmse(real, ip) < thresh):
    iimg.append(x_test[i])
    ilab.append(y_test[i])


aimg = np.array(aimg)
bimg = np.array(bimg)
cimg = np.array(cimg)
dimg = np.array(dimg)
eimg = np.array(eimg)
iimg = np.array(iimg)

alab = np.array(alab)
blab = np.array(blab)
clab = np.array(clab)
dlab = np.array(dlab)
elab = np.array(elab)
ilab = np.array(ilab)

iimg = np.reshape(iimg, [-1, 28, 28, 1])



#[aprd, _] = cl_Hyb.predict(aimg)
[bprd, _] = ae_Hyb.predict(bimg)
[cprd, _] = a_Hyb.predict(cimg)
[dprd, _] = b_Hyb.predict(dimg)
[eprd, _] = c_Hyb.predict(eimg)
iprd = cnn.predict(iimg)


In [ ]:
from sklearn.metrics import roc_curve
for i in range(10):
    #afpr[i], atpr[i], _ = roc_curve(alab[:, i], aprd[:, i])
    bfpr[i], btpr[i], _ = roc_curve(blab[:, i], bprd[:, i])
    cfpr[i], ctpr[i], _ = roc_curve(clab[:, i], cprd[:, i])
    dfpr[i], dtpr[i], _ = roc_curve(dlab[:, i], dprd[:, i])
    efpr[i], etpr[i], _ = roc_curve(elab[:, i], eprd[:, i])
    ifpr[i], itpr[i], _ = roc_curve(ilab[:, i], iprd[:, i])

#afpr["micro"], atpr["micro"], _ = roc_curve(alab.ravel(), aprd.ravel())
bfpr["micro"], btpr["micro"], _ = roc_curve(blab.ravel(), bprd.ravel())
cfpr["micro"], ctpr["micro"], _ = roc_curve(clab.ravel(), cprd.ravel())
dfpr["micro"], dtpr["micro"], _ = roc_curve(dlab.ravel(), dprd.ravel())
efpr["micro"], etpr["micro"], _ = roc_curve(elab.ravel(), eprd.ravel())
ifpr["micro"], itpr["micro"], _ = roc_curve(ilab.ravel(), iprd.ravel())

plt.figure()
for i in range(10):
  #plt.plot(afpr[i], atpr[i],color = 'blue')
  plt.plot(bfpr[i], btpr[i], color = 'forestgreen')
  plt.plot(cfpr[i], ctpr[i], color = 'black')
  plt.plot(dfpr[i], dtpr[i],color = 'red')
  plt.plot(efpr[i], etpr[i], color = 'coral')
  plt.plot(ifpr[i], itpr[i], color = 'blue')  
plt.plot([0,1], [0,1], 'k--')
plt.xlim([-.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

plt.figure()
#plt.plot(afpr["micro"], atpr["micro"], color = 'blue', label = 'Class. Bias')
plt.plot(bfpr["micro"], btpr["micro"], color = 'forestgreen', label = 'Gen. Bias')
plt.plot(cfpr["micro"], ctpr["micro"], color = 'black', label ='25% Class. Bias')
plt.plot(dfpr["micro"], dtpr["micro"], color = 'red', label = '50% Class. Bias')
plt.plot(efpr["micro"], etpr["micro"], color = 'coral', label = '75% Class. Bias')
plt.plot(ifpr["micro"], itpr["micro"], color = 'blue', label ='Independent')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()


plt.figure()
#plt.plot(afpr["micro"], atpr["micro"], color = 'blue', label = 'Class. Bias')
plt.plot(bfpr["micro"], btpr["micro"], color = 'forestgreen', label = 'Gen. Bias')
plt.plot(cfpr["micro"], ctpr["micro"], color = 'black', label ='25% Class. Bias')
plt.plot(dfpr["micro"], dtpr["micro"], color = 'red', label = '50% Class. Bias')
plt.plot(efpr["micro"], etpr["micro"], color = 'coral', label = '75% Class. Bias')
plt.plot(ifpr["micro"], itpr["micro"], color = 'blue', label ='Independent')
plt.legend()
plt.plot([0,1], [0,1], 'k--')
plt.xlim([-0.01, 0.4])
plt.ylim([0.55, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
